In [ ]:
'''
gihanchanaka@gmail.com
08-03-2019
    1)This is to learn from BW and predict random
    2)          learn from random and predict movie
'''

In [ ]:
DEBUG=False
PC=False

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D,Flatten,Dense,Dropout
from tensorflow.contrib.learn.python.learn.estimators._sklearn import train_test_split
import keras.backend as K

import numpy as np
import cv2
import sys
import os
import pandas as pd


In [ ]:
keras.callbacks.TensorBoard(log_dir='./logs/cnnModel04/', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None,)

In [ ]:
FRAME_HEIGHT=100
FRAME_WIDTH=200
COLOR_CHANNELS=3
CELLS_PER_FRAME=25

INPUT_DIM=(FRAME_HEIGHT,FRAME_WIDTH)
OUTPUT_DIM=CELLS_PER_FRAME


EPOCHS=20
BATCH_SIZE=64
CUDA1=0
CUDA2=1

os.environ["CUDA_VISIBLE_DEVICES"]="{},{}".format(CUDA1,CUDA2)

In [ ]:
def cnn(inptuDim,outputDim):
    model=Sequential()
    '''model.add(Conv2D(10, kernel_size=(10,10), activation='relu',strides=(10,10)))
    model.add(Conv2D(5, kernel_size=(2,2), activation='relu',strides=(2,2)))
    model.add(Conv2D(5, kernel_size=(2,2), activation='relu',strides=(2,2)))
    model.add(Conv2D(5, kernel_size=(2,2), activation='relu',strides=(2,2)))
    model.add(Conv2D(5, kernel_size=(2,2), activation='relu',strides=(2,2)))
    model.add(Conv2D(5, kernel_size=(2,2), activation='relu',strides=(2,2)))'''
    model.add(Flatten())
    #model.add(Dense(500,activation='sigmoid'))
    #model.add(Dropout(0.5))
    #model.add(Dense(81,activation='relu'))
    model.add(Dense((CELLS_PER_FRAME),activation='sigmoid'))

    return model


In [ ]:
def makeX(fileNameVideo,noFrames,skipFrames):
    cap = cv2.VideoCapture(fileNameVideo)
    if (cap.isOpened()== False): 
      print("Error opening video stream or file")
    ret,frame=cap.read()
    '''if (len(sys.argv)>1):
        NO_OF_FRAMES=int(sys.argv[1])'''

    X=np.zeros((noFrames,FRAME_HEIGHT,FRAME_WIDTH,COLOR_CHANNELS),dtype=np.float32)
        
    for f in range(NO_FRAMES):
        X[f,:,:,:]=frame
        ret,frame=cap.read()

        '''if f%1 == 0:
            print(len(X[f,:,4,1]))'''
    
    return X[skipFrames:,:,:,:]
#maxeX(cap)

In [ ]:
def makeY(fileNameCsv,noFrames,skipFrames):
    df=pd.read_csv(fileNameCsv, sep=',',header=None)
    df=np.array(df)
    return df[skipFrames:noFrames,:]
    '''df=df[:NO_OF_FRAMES,1]
    
    Y=np.zeros((NO_OF_FRAMES,CELLS_PER_FRAME),dtype=np.int32)
    100
    for f in range(NO_OF_FRAMES):
        if DEBUG: print("DEBUG: df=",df[f])
        for c in range(CELLS_PER_FRAME-1,-1,-1):
            Y[f,c]=int(df[f]/(2**c))
            if Y[f,c]>0:100
                df[f]=df[f]-2**c
        
        if DEBUG: print("DEBUG: Y=",Y[f,:])
    
    return Y'''
    

In [ ]:
def maxDiff(y_true, y_pred):
    return K.max(K.abs(y_true-y_pred))

In [ ]:
model=cnn(INPUT_DIM,OUTPUT_DIM)
model.compile(loss='mean_squared_error',optimizer='adam',metrics=['binary_accuracy','accuracy','mean_squared_error','binary_crossentropy',maxDiff])
#model.predict(xTest,yTest)
#print(model.summary())





In [ ]:
'''
BLACK AND WHITE VIDEO

'''
FILE_NAME='./video/bw'
FILE_NAME_VIDEO=FILE_NAME+'.avi'
FILE_NAME_CSV=FILE_NAME+'.csv'

NO_FRAMES=10000
if PC: NO_FRAMES=100
FRAMES_TO_SKIP=0

In [ ]:
X=makeX(FILE_NAME_VIDEO,NO_FRAMES,FRAMES_TO_SKIP)
Y=makeY(FILE_NAME_CSV,NO_FRAMES,FRAMES_TO_SKIP)

if DEBUG: print("X= ",X)
if DEBUG: print("NAN in X",np.argwhere(np.isnan(X)))
if DEBUG: print("NAN in Y",np.argwhere(np.isnan(Y)))
if DEBUG: print("X shape: ",np.shape(X))
if DEBUG: print("Y shape: ",np.shape(Y))



In [ ]:
'''Preparing the input and output vectors'''
X=np.mean(X,axis=3)
X=np.reshape(X,(X.shape[0],X.shape[1],X.shape[2],1))
X=(X-128.0)/256.0

print("X shape {}, Y shape {}.".format(X.shape,Y.shape))
xTrain, xTest, yTrain, yTest= train_test_split(X, Y, test_size=0.01)

In [ ]:
model.fit(xTrain,yTrain,validation_split=0.1,epochs=EPOCHS,verbose=1,batch_size=BATCH_SIZE,shuffle=True)
print("Test set evaluation for BW video",'\n',model.metrics_names,'\n', model.evaluate(xTest,yTest))

In [ ]:
'''
RANDOM COLOUR VIDEO

'''
FILE_NAME='./video/ran'
FILE_NAME_VIDEO=FILE_NAME+'.avi'
FILE_NAME_CSV=FILE_NAME+'.csv'

NO_FRAMES=10000
if PC: NO_FRAMES=100
FRAMES_TO_SKIP=0

In [ ]:
X=makeX(FILE_NAME_VIDEO,NO_FRAMES,FRAMES_TO_SKIP)
Y=makeY(FILE_NAME_CSV,NO_FRAMES,FRAMES_TO_SKIP)

if DEBUG: print("X= ",X)
if DEBUG: print("NAN in X",np.argwhere(np.isnan(X)))
if DEBUG: print("NAN in Y",np.argwhere(np.isnan(Y)))
if DEBUG: print("X shape: ",np.shape(X))
if DEBUG: print("Y shape: ",np.shape(Y))

INPUT_DIM=X[0].shape
OUTPUT_DIM=CELLS_PER_FRAME

In [ ]:
'''Preparing the input and output vectors'''
X=np.mean(X,axis=3)
X=np.reshape(X,(X.shape[0],X.shape[1],X.shape[2],1))
X=(X-128.0)/256.0

print("X shape {}, Y shape {}.".format(X.shape,Y.shape))
xTrain, xTest, yTrain, yTest= train_test_split(X, Y, test_size=0.10)

In [ ]:
model.fit(xTrain,yTrain,validation_split=0.1,epochs=EPOCHS,verbose=1,batch_size=BATCH_SIZE,shuffle=True)
print("Test set evaluation for random video",'\n',model.metrics_names,'\n', model.evaluate(xTest,yTest))

In [ ]:
'''
MOVIE VIDEO

'''
FILE_NAME='./video/multipleVideos'
FILE_NAME_VIDEO=FILE_NAME+'.mp4'
FILE_NAME_CSV=FILE_NAME+'.csv'

NO_FRAMES=10000
if PC: NO_FRAMES=100
FRAMES_TO_SKIP=0

In [ ]:
X=makeX(FILE_NAME_VIDEO,NO_FRAMES,FRAMES_TO_SKIP)
Y=makeY(FILE_NAME_CSV,NO_FRAMES,FRAMES_TO_SKIP)

if DEBUG: print("X= ",X)
if DEBUG: print("NAN in X",np.argwhere(np.isnan(X)))
if DEBUG: print("NAN in Y",np.argwhere(np.isnan(Y)))
if DEBUG: print("X shape: ",np.shape(X))
if DEBUG: print("Y shape: ",np.shape(Y))

INPUT_DIM=X[0].shape
OUTPUT_DIM=CELLS_PER_FRAME

In [ ]:
'''Preparing the input and output vectors'''
X=np.mean(X,axis=3)
X=np.reshape(X,(X.shape[0],X.shape[1],X.shape[2],1))
X=(X-128.0)/256.0

print("X shape {}, Y shape {}.".format(X.shape,Y.shape))
xTrain, xTest, yTrain, yTest= train_test_split(X, Y, test_size=0.5)

In [ ]:
print("Test set evaluation for Movie video (Without training for movie video): ",model.metrics_names,'  ', model.evaluate(xTest,yTest))
hist= model.fit(xTrain,yTrain,validation_split=0.1,epochs=EPOCHS,verbose=1,batch_size=BATCH_SIZE,shuffle=True)
print("Training history for multiple videos\n",hist.history)
print("Test set evaluation for Movie video (After training for movie video):",model.metrics_names,'  ', model.evaluate(xTest,yTest))

In [ ]:
print(model.summary())

In [1]:
'''
Metrics history on training with multipleVideos.mp4


{'acc': [0.23311111116409303, 0.27755555558204653, 0.3237777777777778, 0.3313333333333333, 0.3271111112170749, 0.3015555555555556, 0.3128888889418708, 0.28600000002649095, 0.2562222222354677, 0.2522222222354677, 0.25133333333333335, 0.23200000001324547, 0.216, 0.20333333333995607, 0.19111111112435658, 0.18222222222884496, 0.1782222222354677, 0.17466666667991215, 0.16977777777777778, 0.1682222222354677], 
'loss': [0.4689356261094411, 0.4327002283467187, 0.38841372656822204, 0.3447624340322283, 0.3095843717257182, 0.28110945081710814, 0.25776883000797696, 0.23929637609587776, 0.22425402291615804, 0.209986483335495, 0.19392411624060737, 0.18129920683966744, 0.16840837666723463, 0.15843162898222604, 0.14632889254887899, 0.138347588552369, 0.12872959072060056, 0.12118555376264784, 0.11496801440583335, 0.10953687610228856], 
'val_binary_accuracy': [0.5310399951934814, 0.5645599946975708, 0.5922399911880493, 0.6363200311660766, 0.6701600122451782, 0.6940800013542175, 0.715200002670288, 0.732000009059906, 0.7495200057029724, 0.7569599866867065, 0.7702399997711181, 0.7800800008773804, 0.7895200023651123, 0.8037599883079529, 0.8120800080299377, 0.825040009021759, 0.8287999973297119, 0.8352000036239624, 0.8403200225830079, 0.848400007724762], 
'val_maxDiff': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 
'val_mean_squared_error': [0.45605633974075316, 0.42020852518081664, 0.39098965883255005, 0.3457904739379883, 0.31442923355102537, 0.289195969581604, 0.26880078673362734, 0.2524260666370392, 0.23652597868442535, 0.22681970071792604, 0.21575466334819793, 0.20458917999267578, 0.19514589083194733, 0.1819763082265854, 0.17384037852287293, 0.16119274771213532, 0.15668206322193146, 0.15043683278560638, 0.14630258786678313, 0.14013233852386475], 
'binary_accuracy': [0.5167377676433987, 0.5532622170978122, 0.5963911172019111, 0.6395288905037774, 0.6762577809227838, 0.7052711139784918, 0.7295022361543444, 0.7492177734904819, 0.765902216858334, 0.7807288846969604, 0.7975733205477397, 0.810782212363349, 0.8242399988174438, 0.8347822304301792, 0.8480444445080227, 0.8562844502131144, 0.86563555182351, 0.8740800044271682, 0.8810933295885722, 0.8873777731789483],
'mean_squared_error': [0.4689356261094411, 0.4327002283467187, 0.38841372656822204, 0.3447624340322283, 0.3095843717257182, 0.28110945081710814, 0.25776883000797696, 0.23929637609587776, 0.22425402291615804, 0.209986483335495, 0.19392411624060737, 0.18129920683966744, 0.16840837666723463, 0.15843162898222604, 0.14632889254887899, 0.138347588552369, 0.12872959072060056, 0.12118555376264784, 0.11496801440583335, 0.10953687610228856], 
'val_acc': [0.27, 0.24400000047683715, 0.3500000007152557, 0.34599999904632567, 0.35199999928474424, 0.30399999952316287, 0.32999999928474427, 0.3079999995231628, 0.21800000035762787, 0.23600000023841858, 0.23200000035762786, 0.21000000071525574, 0.20000000005960464, 0.19000000005960466, 0.18800000071525574, 0.18400000047683715, 0.17800000071525574, 0.17600000059604645, 0.18200000059604646, 0.17200000005960464], 
'binary_crossentropy': [6.0434934344821505, 5.673993946499294, 4.938294124179416, 4.372756219651964, 3.922266551123725, 3.4922371978759768, 3.1899535365634493, 2.937566562652588, 2.7501748347812227, 2.5160920363532173, 2.3125009083218044, 2.117546265072293, 1.9639706023534138, 1.8311143577363755, 1.7072940332624646, 1.5769914582570395, 1.438282094637553, 1.3564661796357897, 1.2955529523425633, 1.2425701642566258], 
'maxDiff': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 
'val_binary_crossentropy': [6.084311828613282, 5.393600395202637, 4.908252456665039, 4.325027175903321, 3.858858102798462, 3.4130125370025635, 3.2056984329223632, 2.9946967754364016, 2.6856933269500733, 2.56715100479126, 2.3758979587554934, 2.2027552070617675, 2.0616827116012573, 1.9408895063400269, 1.8203774585723878, 1.6358685092926026, 1.563847927093506, 1.4947232494354248, 1.450981183052063, 1.3783566598892212], 
'val_loss': [0.45605633974075316, 0.42020852518081664, 0.39098965883255005, 0.3457904739379883, 0.31442923355102537, 0.289195969581604, 0.26880078673362734, 0.2524260666370392, 0.23652597868442535, 0.22681970071792604, 0.21575466334819793, 0.20458917999267578, 0.19514589083194733, 0.1819763082265854, 0.17384037852287293, 0.16119274771213532, 0.15668206322193146, 0.15043683278560638, 0.14630258786678313, 0.14013233852386475]}





'''

"\nMetrics history on training with multipleVideos.mp4\n\n\n{'acc': [0.23311111116409303, 0.27755555558204653, 0.3237777777777778, 0.3313333333333333, 0.3271111112170749, 0.3015555555555556, 0.3128888889418708, 0.28600000002649095, 0.2562222222354677, 0.2522222222354677, 0.25133333333333335, 0.23200000001324547, 0.216, 0.20333333333995607, 0.19111111112435658, 0.18222222222884496, 0.1782222222354677, 0.17466666667991215, 0.16977777777777778, 0.1682222222354677], \n'loss': [0.4689356261094411, 0.4327002283467187, 0.38841372656822204, 0.3447624340322283, 0.3095843717257182, 0.28110945081710814, 0.25776883000797696, 0.23929637609587776, 0.22425402291615804, 0.209986483335495, 0.19392411624060737, 0.18129920683966744, 0.16840837666723463, 0.15843162898222604, 0.14632889254887899, 0.138347588552369, 0.12872959072060056, 0.12118555376264784, 0.11496801440583335, 0.10953687610228856], \n'val_binary_accuracy': [0.5310399951934814, 0.5645599946975708, 0.5922399911880493, 0.6363200311660766, 0.6

In [ ]:
'''

{'trainCost': [0.46859024264288285, 1.8056247896193476e-05, 3.4359163473200137e-08], 'testAcc': [('Testing', 0.5184687487781048), ('Testing', 0.9999218732118607), ('Testing', 1.0)], 'testCost': [0.4695532098412514, 5.327325969590687e-05, 1.7370636615399073e-07], 'trainAcc': [('Training', 0.5185992918538709), ('Training', 0.9999778363721591), ('Training', 1.0)]}
{'trainCost': [0.4123234437896414, 0.1759080855492251, 0.07454100254110267, 0.036741266703576, 0.02083871957753979, 0.013023557303924783, 0.008771657722809518, 0.006294187899155144, 0.004726557264203112, 0.003663805696725809, 0.0029213371587092658, 0.002373888866634138, 0.001977986823467659, 0.0016683598177177378, 0.0014417710809658231, 0.0012500082514338845, 0.0010977049374469837, 0.000974854974258289, 0.0008760972539234185, 0.0007786844467178559], 'testAcc': [('Testing', 0.5458638523794284), ('Testing', 0.787838373214576), ('Testing', 0.901715764953832), ('Testing', 0.946003183058113), ('Testing', 0.96490445714088), ('Testing', 0.9741600357043515), ('Testing', 0.9793789804361428), ('Testing', 0.9824562091736277), ('Testing', 0.9847253196558375), ('Testing', 0.9859394917062893), ('Testing', 0.9871417234657677), ('Testing', 0.9880015922200148), ('Testing', 0.9885191104973957), ('Testing', 0.9890207033248464), ('Testing', 0.9892714968912161), ('Testing', 0.9895262756165425), ('Testing', 0.9897452270149425), ('Testing', 0.9900358285114264), ('Testing', 0.9902189510643102), ('Testing', 0.9903423588746673)], 'testCost': [0.4140078349477926, 0.18620804758967868, 0.08365431869295752, 0.04524826415025505, 0.029184618051264696, 0.021328443376596565, 0.016876004932887236, 0.014380689431575073, 0.012558069952117031, 0.011424249115216124, 0.010504653072281248, 0.009791723025405104, 0.009286434858278104, 0.008890405506382038, 0.008663032266198639, 0.008455174992893152, 0.008295146078108603, 0.00809080660444014, 0.007906229580770347, 0.007811852885042406], 'trainAcc': [('Training', 0.5479326574787148), ('Training', 0.8021082089738636), ('Training', 0.9148276468584025), ('Training', 0.9579975130122523), ('Training', 0.9764068046188896), ('Training', 0.985452204158989), ('Training', 0.990383352891045), ('Training', 0.9932062899549547), ('Training', 0.9949675699286877), ('Training', 0.9961367234238175), ('Training', 0.9969429599959204), ('Training', 0.9975341985474772), ('Training', 0.9979659675619834), ('Training', 0.9982902381381161), ('Training', 0.9985292203136599), ('Training', 0.9987237820323681), ('Training', 0.9988854743524401), ('Training', 0.9990120723727076), ('Training', 0.9991129070372131), ('Training', 0.9992150754562573)]}
{'trainCost': [0.5097371120996113, 0.50219085510773, 0.4983788392966307, 0.4974318659758266, 0.49548106404799447, 0.49477279299422156, 0.49358271345307553, 0.4922049962267091, 0.490216740701772, 0.48905946597268307, 0.487577963101713, 0.48646841207637065, 0.4849474448946458, 0.48428365965432757, 0.4847520367254185, 0.48325968165940875, 0.481964059268372, 0.48171077725253525, 0.48116972740692426, 0.48112548642520664], 'testAcc': [('Testing', 0.49479430310333833), ('Testing', 0.4996518983116633), ('Testing', 0.49983386261553703), ('Testing', 0.5025158203855346), ('Testing', 0.5032357596143892), ('Testing', 0.5031091782111156), ('Testing', 0.5040901910655106), ('Testing', 0.5045253151579748), ('Testing', 0.5052531652058228), ('Testing', 0.5073259480391876), ('Testing', 0.5069778474825847), ('Testing', 0.5078560155403765), ('Testing', 0.508678796170633), ('Testing', 0.5089715175990817), ('Testing', 0.5086392401894436), ('Testing', 0.509414556282985), ('Testing', 0.5105379763283308), ('Testing', 0.5105063285254225), ('Testing', 0.5117325975170618), ('Testing', 0.5122784836382805)], 'testCost': [0.5039619301693349, 0.49904253475273713, 0.49906097484540335, 0.4960763726053359, 0.4954866804654085, 0.49570192830472054, 0.4947258818753158, 0.49425980901416344, 0.49345601585846915, 0.4914801596086236, 0.49164402862138384, 0.4906668945958343, 0.4898064777820925, 0.48963281285913685, 0.48984708023976675, 0.4890519818173179, 0.4879352801208255, 0.48786131792430637, 0.48651415826399114, 0.48624269419078586], 'trainAcc': [('Training', 0.4888370252108272), ('Training', 0.49677215120460416), ('Training', 0.5006012656266177), ('Training', 0.5013765843608712), ('Training', 0.5034098097040683), ('Training', 0.5040901888020431), ('Training', 0.5053876559191113), ('Training', 0.5068987347657168), ('Training', 0.5088291160667999), ('Training', 0.5099604397634917), ('Training', 0.5113607595992994), ('Training', 0.512531643426871), ('Training', 0.5140348114544833), ('Training', 0.5145965180819547), ('Training', 0.5142167719859111), ('Training', 0.5157120250448396), ('Training', 0.5172310109379925), ('Training', 0.5173101274273063), ('Training', 0.5177610731577571), ('Training', 0.5177610780619368)]}




















'''